In [1]:
#!pip freeze --user > requirements.txt
#!while read p; do pip install --user "${p}" --ignore-installed ; done <./requirements.txt

from sklearn.ensemble import RandomForestClassifier

In [2]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_7db59c5e102845a295fa6cc478a0f277(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_4bcd1ecbb4e7d4249f9dadbfc85d90264c24f349','domain': {'id': '8f52255ee4dc4c0db1e290f2433b0de7'},
            'password': 'D0Px8lR.zhuvgPo['}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_2 = pd.read_csv(get_object_storage_file_with_credentials_7db59c5e102845a295fa6cc478a0f277('ENTEL', 'REMOTE_PORTOUT_TRAIN_0.CSV'))
df_data_2.head()


,ID,VAR_1,VAR_2,VAR_3,VAR_4,VAR_5,VAR_6,VAR_7,VAR_8,VAR_9,...,VAR_186,VAR_187,VAR_188,VAR_189,VAR_190,VAR_191,VAR_192,VAR_193,VAR_194,FL_TARGET
0,10003847,3941,1,38,1,1470,19990,0,0,0,...,1.0,1.03,0.93,1.12,0.82,1.00,1.12,1.67,1.67,P
1,10019176,3945,1,38,1,1472,29990,0,0,0,...,1.0,0.96,0.77,0.87,0.77,1.00,0.87,1.00,1.00,P
2,10019424,3945,1,38,1,1472,29990,0,0,0,...,1.0,1.10,0.91,0.91,0.97,1.00,0.91,0.00,0.00,A
3,10022424,3835,3,38,1,1417,6990,0,0,0,...,1.0,1.00,0.99,1.10,1.00,1.00,1.06,1.67,1.67,A
4,10023637,4473,3,38,1,1736,13990,0,0,0,...,1.0,1.05,0.75,1.02,0.70,1.67,1.04,1.00,1.00,P


In [3]:
# 4. Base de train
df = df_data_2
Y_TRAIN = df.FL_TARGET
X_TRAIN = df.drop('FL_TARGET', axis=1) #el target se debe quitar de la muestra
# 4. Divido en train y test
#print X
##X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

In [4]:
#inicializo el modelo
clf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=2,
            oob_score=True, random_state=0, verbose=0, warm_start=False)
#uso el modelo
clf.fit(X_TRAIN, Y_TRAIN) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=2,
            oob_score=True, random_state=0, verbose=0, warm_start=False)

In [5]:
#obtengo el archivo de test
df_test = pd.read_csv(get_object_storage_file_with_credentials_7db59c5e102845a295fa6cc478a0f277('ENTEL', 'REMOTE_PORTOUT_TEST_0.CSV'))
df_test.head()


,ID,VAR_1,VAR_2,VAR_3,VAR_4,VAR_5,VAR_6,VAR_7,VAR_8,VAR_9,...,VAR_186,VAR_187,VAR_188,VAR_189,VAR_190,VAR_191,VAR_192,VAR_193,VAR_194,FL_TARGET
0,10000428,3979,5,40,1,1489,20990,12,17639,12,...,1.00,1.0,1.68,0.64,0.69,0.64,0.78,1.03,1.0,A
1,10000485,4081,1,40,21,1540,9990,0,0,0,...,1.00,1.0,1.00,1.35,1.08,1.36,1.00,1.29,2.0,A
2,1000075,4599,2,162,1,1799,29990,9,17639,6,...,0.22,1.0,0.94,0.90,0.45,1.03,2.00,1.53,1.0,A
3,10001370,4599,6,40,1,1799,29990,17,17639,2,...,1.00,1.0,1.51,1.04,0.00,1.30,1.64,1.43,2.0,A
4,1000199,4081,2,162,1,1540,9990,0,0,0,...,1.00,1.0,1.00,1.23,1.17,1.18,1.00,0.94,2.0,A


In [6]:
#ejecuto el modelo contra la base test
X_TEST = df_test.drop('FL_TARGET', axis=1)
Y_TEST = df_test.FL_TARGET
PRED = clf.predict(X_TEST)
pd.crosstab(Y_TEST, PRED, rownames=['actual'], colnames=['preds'])

preds,A,P
actual,,
A,23447,2838
P,2120,850


In [7]:
#Precision del modelo
print("precisión del modelo: {0: .2f}".format((Y_TEST == PRED).mean()))




precisión del modelo:  0.83
